## Machine Learning Pipelines

In the next two chapters you'll step through every stage of the machine learning pipeline, from data intake to model evaluation. Let's get to it!

At the core of the `pyspark.ml` module are the `Transformer` and `Estimator` classes. Almost every other class in the module behaves similarly to these two basic classes.

`Transformer` classes have a `.transform()` method that takes a DataFrame and returns a new DataFrame; usually the original one with a new column appended. For example, you might use the class `Bucketizer` to create discrete bins from a continuous feature or the class `PCA` to reduce the dimensionality of your dataset using principal component analysis.

Estimator classes all implement a `.fit()` method. These methods also take a DataFrame, but instead of returning another DataFrame they return a model object. This can be something like a `StringIndexerModel` for including categorical data saved as strings in your models, or a `RandomForestModel` that uses the random forest algorithm for classification or regression.

You'll be working to build a model that predicts whether or not a flight will be delayed based on the flights data

In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

from pyspark.sql.types import *
from pyspark.ml import *

sc = SparkContext(master = "local", appName = "Introduction to pySpark") 
spark = SparkSession(sc)
sqlContext = SQLContext(spark)

In [2]:
import pandas as pd

url = 'https://assets.datacamp.com/production/repositories/1237/datasets/fa47bb54e83abd422831cbd4f441bd30fd18bd15/flights_small.csv'
flights = pd.read_csv(url)
flights.head()

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
0,2014,12,8,658.0,-7.0,935.0,-5.0,VX,N846VA,1780,SEA,LAX,132.0,954,6.0,58.0
1,2014,1,22,1040.0,5.0,1505.0,5.0,AS,N559AS,851,SEA,HNL,360.0,2677,10.0,40.0
2,2014,3,9,1443.0,-2.0,1652.0,2.0,VX,N847VA,755,SEA,SFO,111.0,679,14.0,43.0
3,2014,4,9,1705.0,45.0,1839.0,34.0,WN,N360SW,344,PDX,SJC,83.0,569,17.0,5.0
4,2014,3,9,754.0,-1.0,1015.0,1.0,AS,N612AS,522,SEA,BUR,127.0,937,7.0,54.0


In [3]:
url = 'https://assets.datacamp.com/production/repositories/1237/datasets/231480a2696c55fde829ce76d936596123f12c0c/planes.csv'
planes = pd.read_csv(url)
planes.head()

,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,N102UW,1998.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
1,N103US,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
2,N104UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
3,N105UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan
4,N107US,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan


In [4]:
flightSchema = StructType([
    StructField('year', IntegerType(), False),
    StructField('month', IntegerType(), False),
    StructField('day', IntegerType(), False),
    StructField('dep_time', FloatType(), False),
    StructField('dep_delay', FloatType(), False),
    StructField('arr_time', FloatType(), False),
    StructField('arr_delay', FloatType(), False),
    StructField('carrier', StringType(), False),
    StructField('tailnum', StringType(), False),
    StructField('flight', IntegerType(), False),
    StructField('origin', StringType(), False),
    StructField('dest', StringType(), False), 
    StructField('air_time', FloatType(), False), 
    StructField('distance', IntegerType(), False), 
    StructField('hour', FloatType(), False),
    StructField('minute', FloatType(), False)
])

#Create spark_temp from pd_temp
flightsDF = spark.createDataFrame(flights, flightSchema)
flightsDF.show(5)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|   658.0|     -7.0|   935.0|     -5.0|     VX| N846VA|  1780|   SEA| LAX|   132.0|     954| 6.0|  58.0|
|2014|    1| 22|  1040.0|      5.0|  1505.0|      5.0|     AS| N559AS|   851|   SEA| HNL|   360.0|    2677|10.0|  40.0|
|2014|    3|  9|  1443.0|     -2.0|  1652.0|      2.0|     VX| N847VA|   755|   SEA| SFO|   111.0|     679|14.0|  43.0|
|2014|    4|  9|  1705.0|     45.0|  1839.0|     34.0|     WN| N360SW|   344|   PDX| SJC|    83.0|     569|17.0|   5.0|
|2014|    3|  9|   754.0|     -1.0|  1015.0|      1.0|     AS| N612AS|   522|   SEA| BUR|   127.0|     937| 7.0|  54.0|
+----+-----+---+--------+---------+-----

In [5]:
planeSchema = StructType([
    StructField('tailnum', StringType(), False),
    StructField('year', FloatType(), False),
    StructField('type', StringType(), False),
    StructField('manufacturer', StringType(), False),
    StructField('model', StringType(), False),
    StructField('engines', IntegerType(), False),
    StructField('seats', IntegerType(), False),
    StructField('speed', FloatType(), False),
    StructField('engine', StringType(), False)
    ])

#Create spark_temp from pd_temp
planesDF = spark.createDataFrame(planes, planeSchema)
planesDF.show(5)

+-------+------+--------------------+----------------+--------+-------+-----+-----+---------+
|tailnum|  year|                type|    manufacturer|   model|engines|seats|speed|   engine|
+-------+------+--------------------+----------------+--------+-------+-----+-----+---------+
| N102UW|1998.0|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|  NaN|Turbo-fan|
| N103US|1999.0|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|  NaN|Turbo-fan|
| N104UW|1999.0|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|  NaN|Turbo-fan|
| N105UW|1999.0|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|  NaN|Turbo-fan|
| N107US|1999.0|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|  NaN|Turbo-fan|
+-------+------+--------------------+----------------+--------+-------+-----+-----+---------+
only showing top 5 rows



Our model will also include information about the plane that flew the route, so the first step is to join the two tables: flights and planes!

In [6]:
# Rename year column
planesDF = planesDF.withColumnRenamed('year', 'plane_year')

# Join the DataFrames
model_data = flightsDF.join(planesDF, on='tailnum', how="leftouter")

Before you get started modeling, it's important to know that Spark only handles numeric data. That means all of the columns in your DataFrame must be either integers or decimals (called 'doubles' in Spark).

In [7]:
model_data.printSchema()

root
 |-- tailnum: string (nullable = false)
 |-- year: integer (nullable = false)
 |-- month: integer (nullable = false)
 |-- day: integer (nullable = false)
 |-- dep_time: float (nullable = false)
 |-- dep_delay: float (nullable = false)
 |-- arr_time: float (nullable = false)
 |-- arr_delay: float (nullable = false)
 |-- carrier: string (nullable = false)
 |-- flight: integer (nullable = false)
 |-- origin: string (nullable = false)
 |-- dest: string (nullable = false)
 |-- air_time: float (nullable = false)
 |-- distance: integer (nullable = false)
 |-- hour: float (nullable = false)
 |-- minute: float (nullable = false)
 |-- plane_year: float (nullable = true)
 |-- type: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- engines: integer (nullable = true)
 |-- seats: integer (nullable = true)
 |-- speed: float (nullable = true)
 |-- engine: string (nullable = true)



Now you'll use the `.cast()` method to convert all the appropriate columns from your DataFrame `model_data` to integers!

In [8]:
# Cast the columns to integers
model_data = model_data.withColumn("arr_delay", model_data.arr_delay.cast('integer'))
model_data = model_data.withColumn("air_time", model_data.air_time.cast('integer'))
model_data = model_data.withColumn("plane_year", model_data.plane_year.cast('integer'))

You converted just the column `plane_year` to an integer. This column holds the year each plane was manufactured. However, your model will use the planes' age, which is slightly different from the year it was made!

In [9]:
# Create the column plane_age
model_data = model_data.withColumn("plane_age", model_data.year - model_data.plane_year)

## Making a Boolean

Consider that you're modeling a yes or no question: is the flight late? However, your data contains the arrival delay in minutes for each flight. Thus, you'll need to create a boolean column which indicates whether the flight was late or not!

In [10]:
# Create is_late
model_data = model_data.withColumn("is_late", model_data.arr_delay > 0)

# Convert to an integer
model_data = model_data.withColumn("label", model_data.is_late.cast('integer'))

# Remove missing values
model_data = model_data.filter("arr_delay is not NULL and dep_delay is not NULL and \
                               air_time is not NULL and plane_year is not NULL")

You've defined the column that you're going to use as the outcome in your model.
## Strings and factors

Spark requires numeric data for modeling. 

Fortunately, PySpark has functions for handling this built into the `pyspark.ml.features` submodule. You can create what are called 'one-hot vectors' to represent categorical features.

The first step to encoding your categorical feature is to create a `StringIndexer`. Members of this class are Estimators that take a DataFrame with a column of strings and map each unique string to a number. Then, the Estimator returns a Transformer that takes a DataFrame, attaches the mapping to it as metadata, and returns a new DataFrame with a numeric column corresponding to the string column.

The second step is to encode this numeric column as a one-hot vector using a `OneHotEncoder`. This works exactly the same way as the `StringIndexer` by creating an Estimator and then a Transformer. The end result is a column that encodes your categorical feature as a vector that's suitable for machine learning routines!

This may seem complicated, but don't worry! All you have to remember is that you need to create a `StringIndexer` and a `OneHotEncoder`, and the Pipeline will take care of the rest.

In [12]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder

# Create a StringIndexer
carr_indexer = StringIndexer(inputCol='carrier', outputCol='carrier_index')

# Create a OneHotEncoder
carr_encoder = OneHotEncoder(inputCol='carrier_index', outputCol='carrier_fact')

In [13]:
# Create a StringIndexer
dest_indexer = StringIndexer(inputCol='dest', outputCol='dest_index')

# Create a OneHotEncoder
dest_encoder = OneHotEncoder(inputCol='dest_index', outputCol='dest_fact')

## Assemble a vector

The last step in the Pipeline is to combine all of the columns containing our features into a single column. This has to be done before modeling can take place because every Spark modeling routine expects the data to be in this form. You can do this by storing each of the values from a column as an entry in a vector. Then, from the model's point of view, every observation is a vector that contains all of the information about it and a label that tells the modeler what value that observation corresponds to.

Because of this, the `pyspark.ml.feature` submodule contains a class called `VectorAssembler`. This `Transformer` takes all of the columns you specify and combines them into a new vector column.

In [15]:
from pyspark.ml.feature import VectorAssembler
# Make a VectorAssembler
vec_assembler = VectorAssembler(inputCols=["month", "air_time", "carrier_fact", "dest_fact",
                                           "plane_age"], outputCol='features')

## Create the pipeline

Pipeline is a class in the `pyspark.ml` module that combines all the `Estimators` and `Transformers` that you've already created. This lets you reuse the same modeling process over and over again by wrapping it up in one simple object.

In [16]:
# Make the pipeline
flights_pipe = Pipeline(stages=[dest_indexer, dest_encoder
                                , carr_indexer, carr_encoder, vec_assembler])

## Transform the data
Now you're finally ready to pass your data through the Pipeline you created!

In [17]:
# Fit and transform the data
piped_data = flights_pipe.fit(model_data).transform(model_data)

## Test vs Train

After you've cleaned your data and gotten it ready for modeling, one of the most important steps is to split the data into a test set and a train set. After that, don't touch your test data until you think you have a good model! As you're building models and forming hypotheses, you can test them on your training data to get an idea of their performance.

Once you've got your favorite model, you can see how well it predicts the new data in your test set. This never-before-seen data will give you a much more realistic idea of your model's performance in the real world when you're trying to predict or classify new data.

In Spark it's important to make sure you split the data **after** all the transformations. This is because operations like `StringIndexer` don't always produce the same index even when given the same list of strings.

In [18]:
# Split the data into training and test sets
training, test = piped_data.randomSplit([.6, .4])

In [19]:
sc.stop() # close the spark session